In [11]:
from IMPORTS import *
# MountainCar-v0

In [7]:
class Net(nn.Module):
    def __init__(self, env):
        super(Net, self).__init__()

        conv_out_size = env.observation_space.shape[0]
        n_actions = env.action_space.n

        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 256),
            nn.ReLU(),
            nn.Linear(256, n_actions)
        )

    def forward(self, x):
        return self.fc(x)

In [119]:
env = gym.make('MountainCar-v0')

In [121]:
env.observation_space

Box(2,)

In [30]:
net = Net(env)

In [38]:
curr_input = torch.Tensor([[0.25, 0.45],[0.87, 0.15]])

In [32]:
net(curr_input)

tensor([[-0.2620,  0.2332, -0.0692]], grad_fn=<AddmmBackward>)

In [33]:
selector = ptan.actions.ArgmaxActionSelector()
agent = ptan.agent.DQNAgent(action_selector=selector, dqn_model=net)

In [80]:
agent(curr_input)


(array([0, 1]), [None, None])

In [81]:
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=0.5)
agent = ptan.agent.DQNAgent(action_selector=selector, dqn_model=net)

In [82]:
agent(curr_input)

(array([0, 0]), [None, None])

In [83]:
selector.epsilon

0.5

# Experience Source

In [129]:
# ptan.common.wrappers.ScaledFloatFrame()
env = gym.make('MountainCar-v0')
env = ptan.common.wrappers.ScaledFloatFrame(env)
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=0.5)
agent = ptan.agent.DQNAgent(action_selector=selector, dqn_model=net)
exp_source = ptan.experience.ExperienceSource(env=env, agent=agent, steps_count=2)

In [134]:
for exp in exp_source:
    print(exp)
    print()
    if exp[0].done:
        break
    break
    

(Experience(state=array([-0.00207415,  0.        ], dtype=float32), action=1, reward=-1.0, done=False), Experience(state=array([-2.0739976e-03,  1.5618286e-07], dtype=float32), action=2, reward=-1.0, done=False))



## Multiple envs

In [139]:
# env = gym.make('MountainCar-v0')
env1 = ptan.common.wrappers.ScaledFloatFrame(gym.make('MountainCar-v0'))
env2 = ptan.common.wrappers.ScaledFloatFrame(gym.make('MountainCar-v0'))
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=0.0)
agent = ptan.agent.DQNAgent(action_selector=selector, dqn_model=net)
exp_source = ptan.experience.ExperienceSource(env=[env1, env2], agent=agent, steps_count=2)

In [140]:
for indx, exp in enumerate(exp_source):
    print(exp)
    print()
    if indx > 2:
        break

(Experience(state=array([-0.00186803,  0.        ], dtype=float32), action=1, reward=-1.0, done=False), Experience(state=array([-1.8694194e-03, -1.3850575e-06], dtype=float32), action=1, reward=-1.0, done=False))

(Experience(state=array([-0.00179043,  0.        ], dtype=float32), action=1, reward=-1.0, done=False), Experience(state=array([-1.7923838e-03, -1.9585073e-06], dtype=float32), action=1, reward=-1.0, done=False))

(Experience(state=array([-1.8694194e-03, -1.3850575e-06], dtype=float32), action=1, reward=-1.0, done=False), Experience(state=array([-1.8721792e-03, -2.7598307e-06], dtype=float32), action=1, reward=-1.0, done=False))

(Experience(state=array([-1.7923838e-03, -1.9585073e-06], dtype=float32), action=1, reward=-1.0, done=False), Experience(state=array([-1.7962866e-03, -3.9026195e-06], dtype=float32), action=1, reward=-1.0, done=False))



## Exprerience Source First Last

In [166]:
exp_source = ptan.experience.ExperienceSourceFirstLast(env1, agent, gamma=1.0, steps_count=2)

In [181]:
for indx, exp in enumerate(exp_source):
#     print(exp)
#     print()

    if exp.last_state is None:
        print(indx, exp)
        break

198 ExperienceFirstLast(state=array([-2.0439203e-03, -9.0098847e-06], dtype=float32), action=1, reward=-2.0, last_state=None)


# Buffer

In [182]:
buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size=100)

In [188]:
len(buffer)

100

In [192]:
buffer.sample(batch_size=2)

[ExperienceFirstLast(state=array([-1.6526101e-03, -1.9449153e-05], dtype=float32), action=1, reward=-2.0, last_state=array([-1.7002235e-03, -2.5205578e-05], dtype=float32)),
 ExperienceFirstLast(state=array([-1.7946765e-03, -3.2674197e-05], dtype=float32), action=1, reward=-2.0, last_state=array([-1.8655517e-03, -3.6273614e-05], dtype=float32))]

In [205]:
iterator = buffer.experience_source_iter

In [425]:
next(iterator)

ExperienceFirstLast(state=array([-2.0614208e-03, -1.7047838e-06], dtype=float32), action=1, reward=-2.0, last_state=array([-2.0646360e-03, -1.5710714e-06], dtype=float32))

In [426]:
r = exp_source.pop_rewards_steps()

In [427]:
r

[(-200.0, 200),
 (-200.0, 200),
 (-200.0, 200),
 (-200.0, 200),
 (-200.0, 200),
 (-200.0, 200)]

Help on method pop_rewards_steps in module ptan.experience:

pop_rewards_steps() method of ptan.experience.ExperienceSourceFirstLast instance



In [432]:
import enum
class Actions(enum.Enum):
    Skip = 0
    Buy = 1
    Close = 2

In [434]:
a = Actions(1)

In [435]:
a.name

'Buy'

In [436]:
a.value

1

In [437]:
a.Skip

<Actions.Skip: 0>

In [439]:
a.Skip.value

0